In [3]:
!pip install streamlit

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/9b/ea/7219c01b5e92d02d2bc994a36245d99331cd66eb12d284707a2060a013d0/streamlit-1.32.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for altair<6,>=4.0 from https://files.pythonhosted.org/packages/c5/e4/7fcceef127badbb0d644d730d992410e4f3799b295c9964a172f92a469c7/altair-5.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.0.0 from https://files.pythonhosted.org/packages/fa/2a/7f3714cbc6356a0efec525ce7a0613d581072ed6eb53eb7b9754f33db807/blinker-1.7.0-py3-none-any.whl.metadata
  Using cached blinker-1.7.0-py3-none-any.whl.metadata (1.9 kB)
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/fb/2b/a64c2d25a37aeb921fddb929111413049fc5f8b9a4c1aefaffaafe768d54/cachetools-5.3.3-py3-none-any.whl.metadata
  Obtaining dependency information for protobuf<5,>=3.20 from https://files.pythonhosted.org/packages

In [6]:
!pip3 install bot

ERROR: Could not find a version that satisfies the requirement bot (from versions: none)
ERROR: No matching distribution found for bot


In [7]:
import streamlit as st

st.title("College Recommendation")
col1, col2 = st.columns(2)
# with col1:
with st.container():
        # st.markdown(
        #     "Basic excel query tool"
        # )
        name = st.text_input("Name", value="")
        st.subheader("Secondary School Details")
        sub_col1,sub_col2 = st.columns(2)
        with sub_col1:
            sec_p_yr = st.text_input("Passing Year", value="")
        with sub_col2:
            sec_p_prc = st.text_input("Passing Percentage", value="")
        
        st.subheader("Senior Secondary School Details")
        sr_sec_col1,sr_sec_col2 = st.columns(2)
        with sr_sec_col1:
            sr_sec_p_stream= st.text_input("Stream", value="",key="sr_sec_p_stream")
            sr_sec_p_yr = st.text_input("Passing Year", value="",key="sr_sec_p_yr")
        with sr_sec_col2:
            sr_sec_p_prc = st.text_input("Passing Percentage", value="",key="sr_sec_p_prc")

        st.subheader("Graduation Details")
        grd_col1,grd_col2 = st.columns(2)
        with grd_col1:
            grd_p_degree = st.text_input("Degree", value="",key="grd_p_stream")
            grd_p_yr = st.text_input("Passing Year", value="",key="grd_p_yr")
        with grd_col2:
            grd_p_prc = st.text_input("Passing Percentage", value="",key="grd_p_prc")

        st.subheader("Exam Scores")
        exam_col1,exam_col2 = st.columns(2)
        with exam_col1:
            exam_gre = st.text_input("GRE", value="",key="exam_gre")
            exam_pte = st.text_input("PTE", value="",key="exam_pte")
        with exam_col2:
            exam_ielts = st.text_input("IELTS", value="",key="exam_ielts")
            exam_toefl = st.text_input("TOEFL", value="",key="exam_toefl")


        st.subheader("Personal Details")
        psnl_details = st.text_area("Please share carear details, personal story and preferences", value="", height=100, key="psnl_detail", help="Please share details related to carear, life, preferences and others")

        if st.button('Give me recommendations'):
            prompt_input = '''I am  %s, I have completed my undergraduation %s in %s with %s. I have completed my Senior Secondary school in %s with %s from %s background and my Secondary school with %s in %s.
            My GRE score - %s and IELTS score - %s
            My details related to carear, life and college preferences are below:
            %s ''' % (name, grd_p_degree, grd_p_yr, grd_p_prc, sr_sec_p_yr, sr_sec_p_prc, sr_sec_p_stream, sec_p_prc, sec_p_yr,exam_gre, exam_ielts,psnl_details)

# Calling our bot on button click
            st.write(run_flow(prompt_input)['output'])

2024-03-17 20:05:30.533 
  command:

    streamlit run /Users/sanjokdangol/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [8]:
import pandas as pd
from dotenv import load_dotenv
from typing import List,Union
import re
# Importing Langchain modules
from langchain import PromptTemplate, LLMChain
from langchain.agents import create_pandas_dataframe_agent, Tool, AgentExecutor, AgentOutputParser, LLMSingleActionAgent
from langchain.prompts import BaseChatPromptTemplate
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from langchain.llms import OpenAI


load_dotenv()

# Setting LLM
llm = OpenAI(temperature=0)

# Creating Excel agent
df = pd.read_csv("./MS-CS-USA.csv")

excelAgent = create_pandas_dataframe_agent(llm, df, verbose=True)
tools = [
    Tool(
        name="Excel Agent",
        func=excelAgent.run,
        description="Useful for quering data from excel",
    )
]


template = """Answer the following questions as best you can, but speaking as a study abroad counsole might speak. You have access to the following tools:

{tools}

Use the following format:
Question: the input question you must answer {question}
Thought: you can use GRE, IELTS scores and budgets for filtering
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a friend when giving your final answer and share exact information asked by user. Do not add prefix like Human: and AI:.

Question: {input}
{agent_scratchpad}"""   


# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]


prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps", "question"]
)

class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        print("Final output")
        print(llm_output)
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split(
                    "Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output},
                log=llm_output,
            )
            # raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()
tool_names = [tool.name for tool in tools]

# Setting chain and agent
# Chain: Chains allow us to combine multiple components together to create a single, coherent application. For example, we can create a chain that takes user input, formats it with a PromptTemplate, and then passes the formatted response to an LLM. We can build more complex chains by combining multiple chains together, or by combining chains with other components.
# Agents use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning a response to the user.

llm_chain = LLMChain(llm=llm, prompt=prompt)

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    stop=["\nObservation:"],
    output_parser=output_parser,
    allowed_tools=tool_names
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True)


def run_flow(input):
    return agent_executor({'input': input, "question":input})

ImportError: create_pandas_dataframe_agent has been moved to langchain experimental. See https://github.com/langchain-ai/langchain/discussions/11680for more information.
Please update your import statement from: `langchain.agents.create_pandas_dataframe_agent` to `langchain_experimental.agents.create_pandas_dataframe_agent`.